# Terrarium Colab Runner
Run this notebook to execute the Meeting Scheduling simulation in Colab using vLLM.

In [ ]:
# Install GMP and build OT module for privacy-preserving slot intersection
!apt-get update -qq && apt-get install -y libgmp-dev
!pip install -q pybind11 setuptools

!cd crypto && python setup.py install

# Verify OT module
try:
    from crypto import OT_AVAILABLE
    print("✓ OT Module:", "Ready" if OT_AVAILABLE else "Fallback mode")
except:
    print("⚠ OT module not built - using fallback")

In [ ]:
# 1. Projeyi klonla (GitHub URL'in varsa)
!git clone https://github.com/ylmzelff/terrarium.git
%cd terrarium

# VEYA Google Drive'dan kopyala:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r /content/drive/MyDrive/Terrarium /content/
# %cd /content/Terrarium

# 2. Python 3.11 kur (Colab Python 3.10 kullanıyor)
!apt-get update
!apt-get install -y python3.11 python3.11-venv

# 3. Bağımlılıkları kur
!pip install uv
!uv venv --python 3.11 .venv
!source .venv/bin/activate && uv sync

# 4. vLLM kur
!pip install vllm

# 5. MCP server'ı başlat (arka planda)
#!source .venv/bin/activate && python src/server.py &

# 6. Simülasyonu çalıştır
#!source .venv/bin/activate && python examples/base_main.py --config examples/configs/meeting_scheduling.yaml

fatal: destination path 'terrarium' already exists and is not an empty directory.
/content/terrarium
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (4,425 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Source

In [ ]:
!pip install matplotlib

In [ ]:
!pip install matplotlib vllm

In [ ]:
import sys
import os
sys.path.append('/content/terrarium') # Projenin ana dizini hangisiyse

In [ ]:
import subprocess
import time

# Server'ı background process olarak başlat
server_process = subprocess.Popen(
    ["bash", "-c", "source .venv/bin/activate && python src/server.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Server'ın başlamasını bekle
print("MCP server başlatılıyor...")
time.sleep(5)
print("✓ Server hazır!")

MCP server başlatılıyor...
✓ Server hazır!


In [ ]:
!apt-get install python3.11-dev -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.11-dev is already the newest version (3.11.14-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [ ]:
import os

# 1. Önce kütüphaneyi kuralım (Grafik hatası için)
!pip install matplotlib -q

# 2. Dosya içeriğini STRING olarak tanımlayalım (Qwen modeli ve doğru parser ile)
config_text = """

simulation:
  max_iterations: 1
  max_planning_rounds: 1
  max_conversation_steps: 3
  seed: 42
  tags:
    - baseline
  note: "im a note"
environment:
  name: MeetingSchedulingEnvironment
  # Simple meeting scheduling (no CoLLAB dependency)
  num_meetings: 1 # Number of meetings to schedule
  timeline_length: 12 # Not used anymore (kept for compatibility)
  min_participants: 2 # Not used anymore
  max_participants: 4 # Not used anymore
  soft_meeting_ratio: 0.6 # Not used anymore
  # Availability table configuration
  num_days: 1 # Number of days to show in availability table
  slots_per_day: 12 # Number of time slots per day
  intersection: 3 # Number of common available slots (where all agents have 1)
communication_network:
  # Topology used to create blackboards
  # Options: complete | path | star | random
  # random requires `edge_prob` (0..1) and uses `simulation.seed`
  topology: random
  num_agents: 2
  edge_prob: .7
  # consolidate_channels=false => one blackboard per edge
  # consolidate_channels=true => one blackboard per clique (plus leftover edges)
  consolidate_channels: true
llm:
  # Provider: "openai", "anthropic", "gemini", "together", "vllm", or "mock"
  provider: "vllm" # Using Qwen2.5-3B-Instruct on Colab
  openai:
    model: "gpt-4.1-mini-2025-04-14"
    params:
      max_tokens: 1024
      temperature: 0.7
      reasoning_effort: "low"
      verbosity: "low"
  anthropic:
    model: "claude-3-5-sonnet-20241022"
    params:
      max_tokens: 4000
      temperature: 0.7
      top_p: 0.95
  gemini:
    model: "gemini-2.0-flash-lite"
    params:
      max_tokens: 1024
      temperature: 0.6
      top_p: 0.95
  together:
    model: "zai-org/GLM-4.6"
    params:
      max_tokens: 1024
      temperature: 0.7
  vllm:
    auto_start_server: true
    persistent_server: false # Keep the vLLM server alive between runs to avoid cold starts
    health_check_path: "/models"
    startup_timeout: 120 # Increased for Colab (was 420)
    params:
      max_tokens: 512 # Reduced from 1024 to fit in 4096 context window
      temperature: 0.7
    models:
      - checkpoint: "Qwen/Qwen2.5-3B-Instruct"
        host: "127.0.0.1"
        port: 8020
        tensor_parallel_size: 1
        gpu_memory_utilization: 0.70
        max_model_len: 4096
        trust_remote_code: true
        tool_call_parser: "hermes"
        additional_args:
          - "--enforce-eager" # Disable torch.compile to avoid Triton compilation errors
          - "--disable-frontend-multiprocessing"
notes:
  - MeetingScheduling baseline configuration (no attacks)
  - MeetingScheduling always logs scores for each iteration
  - MeetingScheduling environment coordinates attendance intervals across agents
  - Agents coordinate via blackboards during planning phase
  - Final attendance decisions happen during execution using attend_meeting action
  - Meetings have fixed windows on a shared timeline; soft meetings reward overlap
  - Strict meetings reward full‑window attendance; overlaps across two meetings are penalized
  - Objective: Maximize global reward ratio from CoLLAB v2 scoring
  - Simulation stops when all attendance variables are assigned or max iterations reached


"""

# 3. Dosyayı üzerine yazalım (Eski dosyayı tamamen siliyoruz)
config_path = 'examples/configs/meeting_scheduling.yaml'
with open(config_path, 'w') as f:
    f.write(config_text.strip())

print(f"✅ {config_path} güncellendi. Model: Qwen/Qwen2.5-1.5B-Instruct")

# 4. Logları temizle ve çalıştır
!rm -rf logs/vllm/*
!bash -c "source .venv/bin/activate && python examples/base_main.py --config examples/configs/meeting_scheduling.yaml"

✅ examples/configs/meeting_scheduling.yaml güncellendi. Model: Qwen/Qwen2.5-1.5B-Instruct
PersonalAssistant environment not available: No module named 'src.simple_problem_layer'
SmartGrid environment not available: No module named 'src.simple_problem_layer'
PersonalAssistant tools not available: No module named 'src.simple_problem_layer'
SmartGrid tools not available: No module named 'src.simple_problem_layer'
INFO     | Meeting availability: 12 total slots, 3 common slots at indices: [2, 5, 6]
INFO     |   AgentA: 6/12 available (3 common, 3 unique)
INFO     |   AgentB: 6/12 available (3 common, 3 unique)
INFO     | MeetingSchedulingEnvironment initialized with 2 agents
INFO     | Agent Names: AgentA, AgentB
INFO     | Total meetings to schedule: 1
INFO     | Availability table: 1 days x 12 slots/day
INFO     | MCP server environment tools initialization: {'status': 'success', 'message': 'Environment tools initialized for MeetingSchedulingEnvironment'}
INFO     | Initializing communic

In [ ]:
import shutil
from google.colab import files

# Çıktıların olduğu klasörü (örn: 'logs') ziple
# Not: Klasör ismin farklıysa 'logs' yerine onu yaz
shutil.make_archive('simulasyon_sonuclari', 'zip', '/content/terrarium/logs')

# Zip dosyasını bilgisayarına indir
files.download('simulasyon_sonuclari.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Test OT-based private intersection (with fallback support)
try:
    from crypto import compute_private_intersection
    result = compute_private_intersection([2,3,4,6,7,8], [1,3,5,6,7,10])
    print(f"✓ OT Private intersection: {result}")
except ImportError:
    # Fallback: simple intersection
    a, b = set([2,3,4,6,7,8]), set([1,3,5,6,7,10])
    result = sorted(list(a & b))
    print(f"✓ Fallback intersection: {result}")